In [1]:
import numpy as np
import torch
from numpy.testing import assert_array_almost_equal

from pism_emulator.models.pdd import ReferencePDDModel, TorchPDDModel
from pism_emulator.utils import load_hirham_climate_w_std_dev

In [3]:
import numpy as np
from pism_emulator.models.debm import DEBMModel

debm = DEBMModel()

time = np.array([1980.5])
year_fraction = debm.year_fraction(time)


In [29]:
eps = 1e-4
temp = np.array([1e-5, 1, 2])
np.where( temp < eps, 0, temp)

array([0., 1., 2.])

In [5]:
year_fraction = np.array([0.5])
eccentricity = np.array([0.1])
perihelion_longitude = np.array([np.pi/4])
distance_factor_paleo = debm.distance_factor_paleo(year_fraction, eccentricity, perihelion_longitude)


In [30]:
import geopandas

ModuleNotFoundError: No module named 'geopandas'

In [7]:
solar_declination = debm.solar_declination_present_day(year_fraction)

In [8]:
solar_declination

array([0.402769])

In [9]:
obliquity = np.array([np.pi / 4])
solar_longitude = np.array([np.pi * 3 / 4])
solar_declination = debm.solar_declination_paleo(obliquity, solar_longitude)


In [10]:
assertsolar_declination

array([0.55536037])

In [17]:
solar_constant = np.array([1361.0])
distance_factor = np.array([1.1])
hour_angle = np.array([0.8])
latitude = np.array([np.pi / 4])
declination = np.array([np.pi / 8])
insolation = debm.insolation(solar_constant, distance_factor, hour_angle, latitude, declination)


In [19]:
elevation = np.array([1500.0])
debm.atmosphere_transmissivity(elevation)

array([0.698])

In [13]:
np.all?

Signature:       np.all(a, axis=None, out=None, keepdims=<no value>, *, where=<no value>)
Call signature:  np.all(*args, **kwargs)
Type:            _ArrayFunctionDispatcher
String form:     <function all at 0x10421e980>
File:            ~/miniconda3/envs/pism-emulator/lib/python3.11/site-packages/numpy/core/fromnumeric.py
Docstring:      
Test whether all array elements along a given axis evaluate to True.

Parameters
----------
a : array_like
    Input array or object that can be converted to an array.
axis : None or int or tuple of ints, optional
    Axis or axes along which a logical AND reduction is performed.
    The default (``axis=None``) is to perform a logical AND over all
    the dimensions of the input array. `axis` may be negative, in
    which case it counts from the last to the first axis.

    .. versionadded:: 1.7.0

    If this is a tuple of ints, a reduction is performed on multiple
    axes, instead of a single axis or all the axes as before.
out : ndarray, optional
